In [1]:
import tensorflow as tf
import os

In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string("f", "", "kernel")
tf.app.flags.DEFINE_string("dog_dir", "./data/dog", "文件的目录")

In [3]:
# 读取狗图片并转换成张量

filelist= [os.path.join(FLAGS.dog_dir, i) for i in os.listdir(FLAGS.dog_dir)]

#1. 构造文件队列
file_queue = tf.train.string_input_producer(filelist)

# 2. 构造阅读器去读取图片内容（默认读取一张图片）
reader = tf.WholeFileReader()
key, value = reader.read(file_queue)

print(value)

# 3. 对读取的图片数据进行解码
image = tf.image.decode_jpeg(value)

print(image)

# 4. 处理图片大小（统一大小）, resize后，图片数据 uint8 --> float32
image_resize= tf.image.resize_images(image, [200, 200]) 
print(image_resize) 
# 一定要把样本的形状固定 [200, 200,3], 在批处理时要求所有数据形状必须定义
image_resize.set_shape([200,200,3])
print(image_resize)

# 5. 批处理
image_batch = tf.train.batch([image_resize],
                             batch_size=20,
                            num_threads=1,
                            capacity=20)

print(image_batch)

W0204 02:59:45.754449 139641895147328 deprecation.py:323] From <ipython-input-3-fc45f36d98de>:6: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0204 02:59:45.797697 139641895147328 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0204 

Tensor("ReaderReadV2:1", shape=(), dtype=string)
Tensor("DecodeJpeg:0", shape=(?, ?, ?), dtype=uint8)
Tensor("resize/Squeeze:0", shape=(200, 200, ?), dtype=float32)
Tensor("resize/Squeeze:0", shape=(200, 200, 3), dtype=float32)
Tensor("batch:0", shape=(20, 200, 200, 3), dtype=float32)


In [5]:
with tf.Session() as sess:
    #线程协调器
    coord = tf.train.Coordinator()
    
    # 开启读文件的线程
    threads = tf.train.start_queue_runners(sess, coord=coord)
    
    print(sess.run(key))
    print(sess.run(image).shape)
    
    print(sess.run(image_resize))
    coord.request_stop()
    coord.join()

b'./data/dog/dog.18.jpg'
(428, 362, 3)
[[[ 77.        43.        18.      ]
  [ 65.        41.        21.      ]
  [ 37.        25.        13.      ]
  ...
  [207.5      212.5      215.5     ]
  [208.       213.       216.      ]
  [208.5      213.5      216.5     ]]

 [[ 74.26      42.        18.74    ]
  [ 57.15      34.02      13.585   ]
  [ 34.13      21.26      11.87    ]
  ...
  [208.435    213.435    216.435   ]
  [209.       214.       217.      ]
  [211.24     216.24     219.24    ]]

 [[ 71.52      42.74      21.74    ]
  [ 48.04      27.15       8.28    ]
  [ 40.7       29.92      20.18    ]
  ...
  [209.74     214.74     217.74    ]
  [210.74     215.74     218.74    ]
  [210.76     215.76     218.76    ]]

 ...

 [[121.04993  112.43994   94.65991 ]
  [123.744934 114.744934  98.35492 ]
  [126.21997  116.21997  104.21997 ]
  ...
  [ 87.        37.        11.390015]
  [ 80.        33.         7.      ]
  [ 79.21997   36.        11.      ]]

 [[118.04004  110.04004   91.04004 